In [1]:
import json
import asyncio
import time

from aiohttp import ClientSession, BasicAuth
from python_opensky import OpenSky, BoundingBox
from pyensign.ensign import authenticate, publisher, Event

# River
from river import feature_extraction, preprocessing

# Configuration

In [2]:
OPENSKY_CREDS_PATH="opensky-creds.json"
ENSIGN_CREDS_PATH="ensign-creds.json"

def load_opensky_creds():
    with open(OPENSKY_CREDS_PATH) as f:
        creds = json.load(f)
        return BasicAuth(login=creds["username"], password=creds["password"])

OPENSKY_CREDS = load_opensky_creds()

In [3]:
# Continental US
# TODO : update for continental US - (19, 65, -162, -67)
BOUNDING_BOX = BoundingBox(
    min_latitude=30,
    max_latitude=40,
    min_longitude=-120,
    max_longitude=-90,
)

INTERVAL_SEC=10

# Publish Flight Updates

In [4]:
@publisher("flight-positions")
async def get_flights():
    async with ClientSession() as session:
        async with OpenSky(session=session) as opensky:
            await opensky.authenticate(OPENSKY_CREDS)
            
            response = await opensky.get_states(bounding_box=BOUNDING_BOX)
                
            for vector in response.states:
                yield {
                    "icao24": vector.icao24,
                    "callsign": vector.callsign,
                    "origin_country": vector.origin_country,
                    "time_position": vector.time_position,
                    "last_contact": vector.last_contact,
                    "longitude": vector.longitude,
                    "latitude": vector.latitude,
                    "geo_altitude": vector.geo_altitude,
                    "on_ground": vector.on_ground,
                    "velocity": vector.velocity,
                    "true_track": vector.true_track,
                    "vertical_rate": vector.vertical_rate,
                    "sensors": vector.sensors,
                    "barometric_altitude": vector.barometric_altitude,
                    "transponder_code": vector.transponder_code,
                    "special_purpose_indicator": vector.special_purpose_indicator,
                    "position_source": vector.position_source,
                    "category": vector.category,
                }
                
@authenticate(cred_path=ENSIGN_CREDS_PATH)
async def poll_flights():
    for i in range(3):
        async for data in get_flights():
            print(data)
        await asyncio.sleep(INTERVAL_SEC)
            
    # Wait for pending events to get published
    await asyncio.sleep(1)

In [5]:
while True:
    try:
        await poll_flights()
    except Exception as e:
        print(e)
        time.sleep(30)

Timeout occurred while connecting to the OpenSky API
Timeout occurred while connecting to the OpenSky API
Timeout occurred while connecting to the OpenSky API
Timeout occurred while connecting to the OpenSky API
Timeout occurred while connecting to the OpenSky API
Timeout occurred while connecting to the OpenSky API
Timeout occurred while connecting to the OpenSky API
Timeout occurred while connecting to the OpenSky API
Timeout occurred while connecting to the OpenSky API
{'icao24': 'ac96b8', 'callsign': 'AAL1254 ', 'origin_country': 'United States', 'time_position': 1697599860, 'last_contact': 1697599860, 'longitude': -106.5988, 'latitude': 35.0443, 'geo_altitude': None, 'on_ground': True, 'velocity': 8.23, 'true_track': 78.75, 'vertical_rate': None, 'sensors': None, 'barometric_altitude': None, 'transponder_code': None, 'special_purpose_indicator': False, 'position_source': <PositionSource.ADSB: 0>, 'category': <AircraftCategory.LARGE: 4>}
{'icao24': 'aa56d8', 'callsign': 'UAL2199 ',

CancelledError: 